In [ ]:
from amuse.lab import *
from amuse.community.fi.interface import Fi
from amuse.couple.bridge import Bridge
from amuse.ext.star_to_sph import convert_stellar_model_to_SPH
from amuse.ext.orbital_elements import generate_binaries
from amuse.ext.sink import SimpleSinks

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

Roche limit - 10 solar mass bh - 1 solar mass star = 0.010 AU

In [ ]:
# ============================================================
# 0) Helper functions
# ============================================================

def get_blackhole_binary():
    M1 = 100 | units.MSun    # <-- reduce for speed, e.g., 10 MSun
    M2 = 100 | units.MSun    # <-- reduce for speed
    a_bh = 10 | units.AU     # <-- increase for speed (less violent encounter)
    e_bh = 0.0
    bh1, bh2 = generate_binaries(M1, M2, a_bh, e_bh, G=constants.G)
    bhs = Particles()
    bhs.add_particle(bh1)
    bhs.add_particle(bh2)
    bhs.move_to_center()
    return bhs

In [ ]:
# ============================================================
# 1) Build a MESA star
# ============================================================

def make_mesa_star(M=1 | units.MSun, age=0 | units.Myr):      # <-- reduce M for speed
    stellar = MESA()
    star = Particle(mass=M)
    stellar.particles.add_particle(star)
    stellar.evolve_model(age)                                 # <-- increase age for faster structure convergence
    model = stellar.particles[0]
    stellar.stop()
    return model

In [ ]:
# ============================================================
# 2) Split MESA star into point-mass core + SPH envelope
# ============================================================

def make_core_plus_sph_envelope(model, N_sph=5000, M_core_fraction=0.2):
    # N_sph ----> MAIN SPEED PARAMETER
    # Use 500–2000 for laptop testing
    Mtotal = model.mass
    Mcore = Mtotal * M_core_fraction                            # <-- increase to make envelope smaller (faster)

    # --- CORE ---
    core = Particle()
    core.mass = Mcore
    core.radius = 0 | units.RSun
    core.position = [0,0,0] | units.m
    core.velocity = [0,0,0] | units.m/units.s

    # --- ENVELOPE ---
    envelope = convert_stellar_model_to_SPH(
        stellar_model=model,
        number_of_particles=N_sph,                             # <-- reduce to 500–1500 for fast runs
        target_core_mass=Mcore
    )
    envelope.move_to_center()
    core.position = envelope.center_of_mass()

    return core, envelope



In [ ]:
# ============================================================
# 3) Build BH binary + star
# ============================================================

bhs = get_blackhole_binary()

# Lower N_sph for fast mode:
core, gas = make_core_plus_sph_envelope(
    make_mesa_star(1 | units.MSun),
    N_sph=5000                                                # <-- CHANGE THIS FIRST (biggest speed increase)
)

offset = [100, 0, 0] | units.AU                               # <-- increase for smoother/lower-cost interaction
vrel   = [-200, 0, 0] | units.km/units.s                      # <-- reduce magnitude for easier evolution
core.position += offset
gas.position  += offset
core.velocity += vrel
gas.velocity  += vrel



In [ ]:
# ============================================================
# 4) Gravity solver
# ============================================================

converter_grav = nbody_system.nbody_to_si(1 | units.AU, 1 | units.MSun)
gravity = ph4(converter_grav)
gravity.parameters.epsilon_squared = (0.01 | units.AU)**2     # <-- increase to 0.05–0.1 AU for faster/softer forces
gravity.parameters.timestep_parameter = 0.03                   # <-- increase to 0.1 for faster runs

gravity.particles.add_particles(bhs)
gravity.particles.add_particle(core)


In [ ]:
# ============================================================
# 5) Hydrodynamics solver
# ============================================================

converter_hydro = nbody_system.nbody_to_si(1 | units.RSun, 1 | units.MSun)
hydro = Fi(converter_hydro)
hydro.gas_particles.add_particles(gas)
hydro.parameters.use_hydro_flag = True
hydro.parameters.self_gravity_flag = True                     # <-- set False for BIG speed boost
hydro.parameters.gamma = 5.0/3.0
hydro.parameters.timestep = 5e3 | units.s                     # <-- increase to 2e4 for faster runs
hydro.parameters.artificial_viscosity_alpha = 0.5             # <-- increase to 1.0 to stabilize when fast-mode

hydro.evolve_model(0.2 | units.day)                           # <-- reduce to 0.05 day to speed up “relaxation”


In [ ]:
# ============================================================
# 6) SINK ACCRETION
# ============================================================

sink_radius = 1.0 | units.AU                                  # <-- decrease to 0.2 AU for faster, smoother runs
sinks = SimpleSinks(gravity, sink_radius)
sinks.sink_particles.add_particles(bhs)

bh_accreted_mass = 0 | units.MSun
initial_envelope_mass = gas.total_mass()


In [ ]:
# ============================================================
# 7) Bridge coupling
# ============================================================

bridge = Bridge(use_threading=False)
bridge.add_system(gravity, (hydro,))
bridge.add_system(hydro, (gravity,))
bridge.timestep = 0.2 | units.day                              # <-- increase to 1 day for speed (less accuracy)



In [ ]:
# ============================================================
# 8) Animation
# ============================================================

fig = plt.figure(figsize=(8,8))                                # <-- reduce DPI later for speed
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X [AU]')
ax.set_ylabel('Y [AU]')
ax.set_zlabel('Z [AU]')
ax.set_xlim(-400, 400)                                         # <-- shrink these axes for faster drawing
ax.set_ylim(-400, 400)
ax.set_zlim(-100, 100)
ax.set_title('BH binary + MESA star (core + SPH envelope)')

bh_dots, = ax.plot([], [], [], 'o', color='red', markersize=8)
core_dot, = ax.plot([], [], [], 'o', color='orange', markersize=6)
gas_scatter = ax.scatter([], [], [], s=1, color='blue', alpha=0.5)   # <-- reduce s to 0.2 for faster drawing



In [ ]:
# ============================================================
# 9) Update function
# ============================================================

t_end = 80 | units.day                                         # <-- reduce to 10 days for fast mode
dt = 0.5 | units.day                                           # <-- increase to 1.0–2.0 for speed
frames = int(t_end/dt)

def init():
    gas_scatter._offsets3d = ([], [], [])
    return bh_dots, core_dot, gas_scatter

def update(frame):
    global bh_accreted_mass

    time = (frame + 1) * dt
    bridge.evolve_model(time)

    # --- SINK ACCRETION ---
    dm = sinks.accreted_mass
    bh_accreted_mass += dm
    sinks.accreted_mass = 0 | units.MSun

    # --- MASS LOSS CONDITION ---
    current_mass = hydro.gas_particles.total_mass()
    frac_lost = (initial_envelope_mass - current_mass) / initial_envelope_mass

    if frac_lost > 0.5:                                        # <-- lower threshold to stop earlier for testing
        print(f"\nStop: Envelope lost {frac_lost*100:.1f}%")
        print(f"BH accreted mass: {bh_accreted_mass.in_(units.MSun)}")
        anim.event_source.stop()
        return bh_dots, core_dot, gas_scatter

    # --- POSITIONS ---
    bhs_pos = gravity.particles[:2].position.value_in(units.AU)
    core_pos = gravity.particles[-1].position.value_in(units.AU)
    gas_pos = hydro.gas_particles.position.value_in(units.AU)

    # Minimal updates to speed up animation
    bh_dots.set_data(bhs_pos[:,0], bhs_pos[:,1])
    bh_dots.set_3d_properties(bhs_pos[:,2])
    core_dot.set_data([core_pos[0][0]], [core_pos[0][1]])
    core_dot.set_3d_properties([core_pos[0][2]])
    gas_scatter._offsets3d = (gas_pos[:,0], gas_pos[:,1], gas_pos[:,2])

    ax.set_title(f"t = {time.value_in(units.day):.1f} days")
    return bh_dots, core_dot, gas_scatter



In [ ]:
# ============================================================
# 10) Run animation
# ============================================================

anim = FuncAnimation(
    fig, update, frames=frames, init_func=init,
    interval=100, blit=False, repeat=False
)

anim.save('bh_star_core_envelope_sink.mp4',
          writer='ffmpeg', fps=20, dpi=150)                    # <-- reduce dpi to 80 for faster rendering

plt.show()


# Cleanup
bridge.stop()
gravity.stop()
hydro.stop()